<a href="https://colab.research.google.com/github/tryingworks/CHE3023/blob/master/StefanMaxwell_NRTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

#### Using the Non-Random Two Liquid (NRTL) activity coefficient, one can calculate the activity coefficient of a component $\gamma_i$ in a mixture by:

$$\ln{\gamma_i}=\dfrac{\sum\limits_{j=1}^n x_j \tau_{ji} G_{ji}}{\sum\limits_{k=1}^n {x_k G_{ki}}} + 
\sum\limits_{j=1}^n 
\dfrac {x_j G_{ij}}{\sum\limits_{k=1}^{n}x_k G_{kj}}
\left(\tau_{ij}-
\dfrac{\sum\limits_{m=1}^n x_m \tau_{mj} G_{mj}}
{\sum\limits_{k=1}^n x_k G_{kj}}
\right)
$$

$$\begin{align}
G_{ij} &= \exp \left(-\alpha_{ij}\tau_{ij} \right) \\
\tau_{ij} &= A_{ij} + \dfrac{B_{ij}}{T} + C_{ij}\ln{T} + D_{ij}T \\
\alpha_{ij} &= \alpha_{ji}
\end{align}$$


here, $x$ is a vector of mole fractions while $\tau$ and $G$ are matrices calculated from $A$, $B$, $C$, $D$, and $\alpha$ that are matrices of binary interaction parameters determined experimentally.  Unlike the other matrices, the $\alpha$ interaction matrix is symmetric.

In [0]:
# For the 3 component mixture: (1) Methanol, (2) Ethanol, and (3) Water system, 
# the matrices A, C, D are zeros, while B and alpha have been determined experimentally.

B=tf.constant( [[   0.,   -155.5,   -24.49],
 [ 190.06,    0.,    -55.17],
 [ 307.17,  670.44,    0.  ]])


alpha = tf.constant( [[0.,    0.305, 0.3  ],
 [0.305, 0.,    0.303],
 [0.3,   0.303, 0.   ]])

In [0]:
@tf.function
def ln_gamma(x,T):
  tau = B/T
  G = tf.exp(-alpha*tau)
  xtauG = tf.einsum('j, ji, ji -> i', x, tau, G)
  xG = tf.einsum('k,ki -> i', x, G)
  xtauGdivxG = xtauG/xG
  return xtauGdivxG + tf.einsum('j,ij,ij -> i', x/xG, G, tau - tf.broadcast_to(xtauGdivxG, (3,3)))

In [0]:
x12=tf.Variable([0.2, 0.3])
T=300
with tf.GradientTape() as g:
  g.watch(x12)
  x3=tf.Variable([1- tf.reduce_sum(x12)])
  x=tf.concat((x12, x3), axis=0)
  ln_g = ln_gamma(x, T)
dln_g_dx = g.jacobian(ln_g,x12)

In [0]:
#thermodynamic factor matrix (Eq 1.31 and 1.32 - Principles and Modern Applications of Mass Transfer Operations
#2nd Edition, Benitez)
tfm = tf.eye(2)+tf.broadcast_to(x12,(2,2))*dln_g_dx[:2,:]

In [0]:
#driving force
dxdz = tf.Variable([0.3, 0.4])
d = tf.linalg.matvec(tfm, dxdz)
d

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.25761628, 0.28023466], dtype=float32)>